In [ ]:
import numpy as np
from numpy import sin, cos,exp,pi
from time import process_time, time
from scipy.optimize import curve_fit, least_squares
from scipy.fft import fft,fftshift,fft2, fftfreq, ifft
from scipy.signal import oaconvolve, medfilt2d

from scipy.ndimage import rotate
from scipy.interpolate import interp1d

import h5py
from skimage.transform import radon
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib import transforms
from PIL import Image, ImageFilter
import joblib as jb
import tifffile as tfl
import os
from IPython.display import clear_output
import re
import pandas as pd

In [ ]:
%run "C:\Users\gabri\Documents\Python Scripts\NPGMI-Library\NPGMI Base Code.ipynb"

### Sample Density Plots

In [ ]:

p = 2e-6
L = 8
Ls = 3

mt = 5

lam = np.array([5e-10])
plam = np.array([1])

polyspectrum = pd.read_excel("ng6_MCNP_spectrum.xlsx").to_numpy().T
polyspectrum[0]*=1e-9

lam, plam = polyspectrum.copy()


x0 = np.array([0])


Nbc = 5e28*4.149e-15


phi1 = phi2 = phi3 = Nbc*lam * np.pi/2 / (Nbc*lam[np.argmax(plam)]) 



camsize = 2.5e-2
slitx = 200e-6

xbin = 1e-6
ybin = 1e-6

res = 100e-6
L1 = 4


dsample = np.linspace(0.25,4.25,161) * p**2/lam[np.argmax(plam)]

ps_vals = np.linspace(0.1,3,101) * p


# Ls_vals = np.linspace(1,3,3)



In [ ]:
t0 = time()



# mt = 5

# G1sd = {"p":p, "L1": np.array([L1]), "phi": np.array([phi1])}
# G1sd["L2"] = dsample


# G2sd = {"p":p, "L1": G1sd["L2"] + G1sd["L1"], "phi": np.array([phi2])}


# G2sd["L2"] = np.array([L]) - G2sd["L1"]



# simaptsdens = {"G1":RectGrating(G1sd),"G2":RectGrating(G2sd)}

# siminit = {"lam":lam, "plam":plam, "L":L,"d": dsample  ,"mt":mt, "x": np.linspace(0,camsize,int(camsize/xbin) + 1),"x0" : x0,"xbin":xbin,"ybin":ybin ,"pos":np.array([[0],[1]]),\
#            "camsize" : camsize ,"convmode" : "valid", "slitx": slitx, "res": res, "batches" : 1000  }



# simdens = PGMI(simaptsdens,siminit)

# simdens.propagate_to("G1")
# simdens.propagate_to("G2")

# simdens.generate_after("G2")



# plt.plot(dsample*1e3,simdens.contrast)
# plt.show()

# contnosamplemono = np.squeeze(simdens.contrast)


lam, plam = polyspectrum.copy()


G1sd = {"p":p, "L1": np.array([L1]), "phi": np.array([phi1])}
G1sd["L2"] = dsample


G2sd = {"p":p, "L1": G1sd["L2"] + G1sd["L1"], "phi": np.array([phi2])}


G2sd["L2"] = np.array([L]) - G2sd["L1"]
simaptsdens = {"G1":RectGrating(G1sd),"G2":RectGrating(G2sd)}

siminit = {"lam":lam, "plam":plam, "L":L,"d": dsample  ,"mt":mt, "x": np.linspace(0,camsize,int(camsize/xbin) + 1),"x0" : x0,"xbin":xbin,"ybin":ybin ,"pos":np.array([[0],[1]]),\
           "camsize" : camsize ,"convmode" : "valid", "slitx": slitx, "res": res, "batches" : 1000  }

simdens = PGMI(simaptsdens,siminit)

simdens.propagate_to("G1")
simdens.propagate_to("G2")

simdens.generate_after("G2")



plt.plot(dsample*1e3,simdens.contrast)
plt.show()

contnosamplepoly = np.squeeze(simdens.contrast)



print(time() - t0, "seconds")

In [ ]:
t0 = time()

# contsamplepgregime = []





G1sd = {"p":p, "L1": np.array([L1]), "phi": np.array([phi1])}
G1sd["L2"] = dsample


G2sd = {"p":p, "L1": G1sd["L2"] + G1sd["L1"], "phi": np.array([phi2])}


G2sd["L2"] = np.array([L]) - Ls - G2sd["L1"]


mt = 5


# for i, psample in enumerate(ps_vals):

psize = 10*ps_vals

profile = grating_equation(np.linspace(0,psize,1001),ps_vals, 0).T



G3sd = {"p":ps_vals, "L1": G2sd["L2"] + G2sd["L1"], "phi": np.array([phi3]), "mt":mt, "spectrum_spacing":1}
G3sd["L2"] = np.array([L]) - G3sd["L1"] 





simaptsdens = {"G1":RectGrating(G1sd),"G2":RectGrating(G2sd),"G3":Sample(G3sd,psize,profile)}





siminit = {"lam":lam, "plam":plam, "L":L,"d": dsample  ,"mt":mt, "x": np.linspace(0,camsize,int(camsize/xbin) + 1),"x0" : x0,"xbin":xbin,"ybin":ybin,\
           "pos":np.array([[0],[1]]), "camsize" : camsize ,"convmode" : "valid", "slitx": slitx, "res": res, "batches" : 6250  }

simdens = PGMI(simaptsdens,siminit)

simdens.propagate_to("G1")
simdens.propagate_to("G2")
simdens.propagate_to("G3")
simdens.generate_after("G3")

# clear_output(wait = True)
# print((i+1)/len(ps_vals)*1e2, "% done")

# contsamplepgregime.append(simdens.contrast)
# del simdens



    # plt.plot(dsample*1e3,simdens.contrast)
    # plt.show()

    
print(time() - t0, "seconds")
# ps_vals = np.linspace(L*lam/(4*p), 5*L*lam/(4*p), 51)


In [ ]:
contsamplepgregime = np.squeeze(contsamplepgregime)
print(contsamplepgregime.shape)
# np.save("monocontsamplepgregime.npy",contsamplepgregime)
# contsamplepgregime = np.reshape(contsamplepgregime, (2,2,len(ps_vals), len(dsample)))

In [ ]:
# lowcontmask = contnosample5ord < 5e-2
# contsamplepgregime = np.squeeze(contsamplepgregime)
# cont5ordmasked = contsamplepgregime[-1]
# cont5ordmasked[...,lowcontmask] = 0

fig, ax = plt.subplots(figsize = (12,8))
c = ax.imshow(contsamplepgregime.T / contnosample5ord, aspect = "auto", origin = "lower", extent = (dsample[0], dsample[-1], ps_vals[0],ps_vals[-1]), clim = (0,1))

fig.colorbar(c)
ax.set_xlabel("$d$ [mm]")
ax.set_ylabel("Sample period [$\mu m$]")

# fig.savefig("contsample5ordpolypgregime.jpg")

# ax.axvline(2*p**2/lam[np.argmax(plam)], color = "red")
# ax.axvline(4*p**2/lam[np.argmax(plam)], color = "red")

# ax.ax.plot(dsample, 2*lam*dsample*Ls/(p*L)/2)
# ax.plot(dsample*1e3, dsample*Ls*1e3/30)

# ax.plot(dsample, 2*lam*dsample*Ls/(p*L) )
# ax.plot(dsample, 2*lam*dsample*Ls/(p*L)/2, "--")

# ax.plot(dsample, 2*lam*dsample*Ls/(p*L)/3)
# ax.plot(dsample, 2*lam*dsample*Ls/(p*L)/4, "--")

# ax.plot(dsample, 2*lam*dsample*Ls/(p*L)/5)


plt.show()
plt.close(fig)

In [ ]:
fig, ax = plt.subplots(figsize = (12,8))
c = ax.imshow(contsamplepmregime[-1][0] / contnosample5ord, aspect = "auto", origin = "lower",  extent = np.squeeze([dsample[0]*1e3, dsample[-1]*1e3, ps_vals[0].squeeze()*1e6,ps_vals[-1].squeeze()*1e6]), clim = (0.9,1))
fig.colorbar(c)

ax.set_xlabel("$d$ [mm]")
ax.set_ylabel("Sample period [$\mu m$]")
ax.axvline(2*p**2/lam * 1e3, color = "red")
ax.axvline(4*p**2/lam * 1e3, color = "red")

# fig.savefig("samplehasmoireperiod.jpg")
plt.show()
plt.close(fig)

### Phi_3 Sample Plots

In [ ]:
t0 = time()


phi3_vals = np.pi * np.linspace(0,1,9)

lam = np.array([5e-10])
plam = np.array([1])


G1sd = {"p":p, "L1": np.array([L1]), "phi": np.array([phi1])}
G1sd["L2"] = dsample


G2sd = {"p":p, "L1": G1sd["L2"] + G1sd["L1"], "phi": np.array([phi2])}


G2sd["L2"] = np.array([L]) - Ls - G2sd["L1"]

psize = 10*p

profile = grating_equation(np.linspace(0,psize,1001),p, 0)

# for phi3 in phi3_vals:
    

G3sd = {"p":p, "L1": G2sd["L2"] + G2sd["L1"], "phi": phi3_vals, "mt":mt, "spectrum_spacing":1}
G3sd["L2"] = np.array([L]) - G3sd["L1"] 





simaptsdens = {"G1":RectGrating(G1sd),"G2":RectGrating(G2sd),"G3":Sample(G3sd,psize,profile)}




siminit = {"lam":lam, "plam":plam, "L":L,"d": dsample  ,"mt":mt, "x": np.linspace(0,camsize,int(camsize/xbin) + 1),"x0" : x0,"xbin":xbin,"ybin":ybin,\
           "pos":np.array([[0],[1]]), "camsize" : camsize ,"convmode" : "valid", "slitx": slitx, "res": res, "batches" : 12500  }

simdens = PGMI(simaptsdens,siminit)

simdens.propagate_to("G1")
simdens.propagate_to("G2")
simdens.propagate_to("G3")
simdens.generate_after("G3")


# clear_output(wait = True)

# print((i+1)/len(ps_vals)*1e2, "% done")


contsamplephi3mono = np.squeeze(simdens.contrast)
    
    
    
# contsamplephi3poly = []

# lam, plam = polyspectrum.copy()


# G1sd = {"p":p, "L1": np.array([L1]), "phi": np.array([phi1])}
# G1sd["L2"] = dsample


# G2sd = {"p":p, "L1": G1sd["L2"] + G1sd["L1"], "phi": np.array([phi2])}


# G2sd["L2"] = np.array([L]) - Ls - G2sd["L1"]

# psize = 10*p

# profile = grating_equation(np.linspace(0,psize,1001),p, 0)

# # for phi3 in phiprime_vals:
    

# G3sd = {"p":p, "L1": G2sd["L2"] + G2sd["L1"], "phi": np.array([phi3_vals]), "mt":mt, "spectrum_spacing":1}
# G3sd["L2"] = np.array([L]) - G3sd["L1"] 





# simaptsdens = {"G1":RectGrating(G1sd),"G2":RectGrating(G2sd),"G3":Sample(G3sd,psize,profile)}




# siminit = {"lam":lam, "plam":plam, "L":L,"d": dsample  ,"mt":mt, "x": np.linspace(0,camsize,int(camsize/xbin) + 1),"x0" : x0,"xbin":xbin,"ybin":ybin,\
#            "pos":np.array([[0],[1]]), "camsize" : camsize ,"convmode" : "valid", "slitx": slitx, "res": res, "batches" : 12500  }

# simdens = PGMI(simaptsdens,siminit)

# simdens.propagate_to("G1")
# simdens.propagate_to("G2")
# simdens.propagate_to("G3")
# simdens.generate_after("G3")


# clear_output(wait = True)
# print((i+1)/len(ps_vals)*1e2, "% done")

# contsamplepgregime1dprime.append(simdens.contrast)
# del simdens

# contsamplephi3poly = simdens.contrast.squeeze()


    
print(time() - t0, "seconds")



In [ ]:
fig, ax1 = plt.subplots(figsize  = (12,8))


[ax1.plot(dsample, contsamplepgregime1d[i] / contnosample, label = "phi = %f" % phi ) for i, phi in enumerate(phi3_vals)]
ax1.set_prop_cycle(None)
[ax1.plot(dsample, contsamplepgregime1dprime[i] / contnosample, "--", label = "phi = %f" % phi ) for i, phi in enumerate(phiprime_vals)]

ax1.legend()
plt.show()
plt.close(fig)

In [ ]:
t0 = time()

contsamplepmregime1d = []





G1sd = {"p":p, "L1": np.array([L1]), "phi": np.array([phi1])}
G1sd["L2"] = dsample


G2sd = {"p":p, "L1": G1sd["L2"] + G1sd["L1"], "phi": np.array([phi2])}


G2sd["L2"] = np.array([L]) - Ls - G2sd["L1"]

psample = 1e-3


psize = 10*psample

profile = grating_equation(np.linspace(0,psize,1001),psample, 0)

for phi3 in phi3_vals:
    

    G3sd = {"p":psample, "L1": G2sd["L2"] + G2sd["L1"], "phi": np.array([phi3]), "mt":mt, "spectrum_spacing":1}
    G3sd["L2"] = np.array([L]) - G3sd["L1"] 





    simaptsdens = {"G1":RectGrating(G1sd),"G2":RectGrating(G2sd),"G3":Sample(G3sd,psize,profile)}




    siminit = {"lam":lam, "plam":plam, "L":L,"d": dsample  ,"mt":mt, "x": np.linspace(0,camsize,int(camsize/xbin) + 1),"x0" : x0,"xbin":xbin,"ybin":ybin,\
               "pos":np.array([[0],[1]]), "camsize" : camsize ,"convmode" : "valid", "slitx": slitx, "res": res, "batches" : 1000  }

    simdens = PGMI(simaptsdens,siminit)

    simdens.propagate_to("G1")
    simdens.propagate_to("G2")
    simdens.propagate_to("G3")
    simdens.generate_after("G3")
    clear_output(wait = True)
    # print((i+1)/len(ps_vals)*1e2, "% done")

    contsamplepmregime1d.append(simdens.contrast)
    del simdens

    
    
    
contsamplepmregime1dprime = []



G1sd = {"p":p, "L1": np.array([L1]), "phi": np.array([phi1])}
G1sd["L2"] = dsample


G2sd = {"p":p, "L1": G1sd["L2"] + G1sd["L1"], "phi": np.array([phi2])}


G2sd["L2"] = np.array([L]) - Ls - G2sd["L1"]



for phi3 in phiprime_vals:
    

    G3sd = {"p":psample, "L1": G2sd["L2"] + G2sd["L1"], "phi": np.array([phi3]), "mt":mt, "spectrum_spacing":1}
    G3sd["L2"] = np.array([L]) - G3sd["L1"] 





    simaptsdens = {"G1":RectGrating(G1sd),"G2":RectGrating(G2sd),"G3":Sample(G3sd,psize,profile)}




    siminit = {"lam":lam, "plam":plam, "L":L,"d": dsample  ,"mt":mt, "x": np.linspace(0,camsize,int(camsize/xbin) + 1),"x0" : x0,"xbin":xbin,"ybin":ybin,\
               "pos":np.array([[0],[1]]), "camsize" : camsize ,"convmode" : "valid", "slitx": slitx, "res": res, "batches" : 1000  }

    simdens = PGMI(simaptsdens,siminit)

    simdens.propagate_to("G1")
    simdens.propagate_to("G2")
    simdens.propagate_to("G3")
    simdens.generate_after("G3")
    clear_output(wait = True)
    # print((i+1)/len(ps_vals)*1e2, "% done")

    contsamplepmregime1dprime.append(simdens.contrast)
    del simdens




    
print(time() - t0, "seconds")

In [ ]:
fig, ax1 = plt.subplots(figsize  = (12,8))
ax1.plot(dsample, contnosample5ord, "o-")

[ax1.plot(dsample, contsamplepmregime1d[i] / contnosample5ord, label = "phi = %f" % phi ) for i, phi in enumerate(phi3_vals)]
ax1.set_prop_cycle(None)
[ax1.plot(dsample, contsamplepmregime1dprime[i] / contnosample5ord, "--", label = "phi = %f" % phi ) for i, phi in enumerate(phiprime_vals)]

ax1.legend()

plt.ylim(0,1)
plt.show()
plt.close(fig)